In [ ]:
#!pip install import-ipynb

In [3]:
import import_ipynb
from RetinaUtils import Retina
from DataLoaders import get_dataloader

import numpy as np
import cv2
import zmq
import time
import pickle
from matplotlib import pyplot as plt
import zlib

import torch

importing Jupyter notebook from RetinaUtils.ipynb
importing Jupyter notebook from DataLoaders.ipynb


In [4]:
#Load Piotr's Retina
gpu = torch.cuda.is_available()
R = Retina(gpu=gpu)
R.loadLoc('../retina_data/retinas/ret50k_loc.pkl')
R.loadCoeff('../retina_data/retinas/ret50k_coeff.pkl')
fixation = (1080/2,1920/2)
shape = (1080, 1920)
R.prepare(shape, fixation)

In [105]:
#Get images
batch_size = 8
dataloader = get_dataloader(image_type='original', personal=True, skip=800, batch_size=batch_size)
#Determine amount of images
dataset = dataloader['test']
dataset_length = len(dataset)

In [125]:
#Connect to the server
context = zmq.Context()
socket = context.socket(zmq.PUSH)
socket.connect("tcp://localhost:5555")
#Send dataset size
socket.send_string(str(dataset_length))

#Iterate over dataloader
for batch, labels in dataset:
    vectors = []
    #Piotr's Retina only works with single Numpy arrays, so we need to sample
    #each image individually
    for img in batch:
        img = img.permute(1,2,0).numpy()*255
        V = R.sample(img, fixation).astype('uint8')
        vectors.append(V)
    
    #Compress and send the retinal vectors and labels 
    data = pickle.dumps([vectors, labels], protocol=-1)
    data = zlib.compress(data, 6)
    socket.send(data)